In [1]:
from twitterscraper import query_tweets
from datetime import datetime
from pandas import DataFrame

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'}


## Choose arguments

In [2]:
datetime.today().strftime('%Y-%m-%d')

'2020-04-28'

In [3]:
# advanced query
location =  None

radius = None

#terms = [['brood', 'sneetjes', 'broodbakmachine'], ['homemade', 'zelfgemaakt', 'zelfgebakken']]
#terms = [['tendeuse', 'tondeuse', 'kaal', 'scheren', 'bald', 'peaky cut', 'corona cut']]
#terms = [['corona maatregelen', 'corona boete', 'social distancing', 'corona regels']]
terms = [['toiletpapier', 'toilet papier', 'wc papier', 'wc-papier']]


# limit (should always be None for our case)
limit = None

## begin and end date are very import to us
begindate = '2019-09-01'
enddate = datetime.today().strftime('%Y-%m-%d')

lang = 'nl'

query = ''
for id1, and_group in enumerate(terms):
    if id1 == 0:
        query += '('
    else:
        query += ' AND ('
    query += ' OR '.join(and_group)
    query += ')'

if location:
    query += ' near:{}'.format(location)
if radius:
    query += ' within:{}'.format(radius)
if lang:
    query += ' lang:{}'.format(lang)
if begindate:
    query += ' since:{}'.format(begindate)
if enddate:
    query += ' until:{}'.format(enddate)

print(query)


(toiletpapier OR toilet papier OR wc papier OR wc-papier) lang:nl since:2019-09-01 until:2020-04-28


In [4]:
def valid_date(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d").date()
    except ValueError:
        print("Not a valid date: {}".format(s))

begindate = valid_date(begindate)
enddate = valid_date(enddate)

## Search Tweets

In [5]:
tweets = query_tweets(query= query, limit=limit, begindate=begindate, enddate=enddate, poolsize=1, lang=lang)

oiletpapier%20OR%20toilet%20papier%20OR%20wc%20papier%20OR%20wc-papier)%20lang%3Anl%20since%3A2019-09-01%20until%3A2020-04-28%20since%3A2019-09-01%20until%3A2020-04-28&l=nl
INFO: Using proxy 111.95.23.43:3128
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgLChod2vsiIWkMC9gfCMh-oiEjUAFQAlAFUAFQAA&q=(toiletpapier%20OR%20toilet%20papier%20OR%20wc%20papier%20OR%20wc-papier)%20lang%3Anl%20since%3A2019-09-01%20until%3A2020-04-28%20since%3A2019-09-01%20until%3A2020-04-28&l=nl
INFO: Using proxy 88.99.10.251:1080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgLqdwviMsiIWkMC9gfCMh-oiEjUAFQAlAFUAFQAA&q=(toiletpapier%20OR%20toilet%20papier%20OR%20wc%20papier%20OR%20wc-papier)%20lang%3Anl%20since%3A

In [6]:
for tweet in tweets:
    print(tweet.text)

 daarom is het handig als de verkoper iets meer uitlegt aan de deur. Dan alleen maar zeggen dat ie toilet papier verkoopt voor Sierra Leone.  Maar fijn dat het allemaal goed is. Ik hoop dat de man veel verkoopt 
Iemand aan de deur die toilet papier van de action verkoopt voor kinderen in Sierra Leone? Raar verhaal @POL_Kampen ?
Tijd voor actie: https://www.youtube.com/watch?v=WBx_0e5jagw …
Toiletpapier met een missie! Het toiletpapier is van 100% gerecycled Pools oud papier, verkrijgbaar bij de buurtwinkel in Warszawa. Zal ze leren.  pic.twitter.com/kQpvpmxCox
Amerikaans onderzoek heeft uitgewezen dat veel toiletpapier ongebruikt wordt doorgespoeld. Dit papier wordt tegenwoordig apart ingezameld en gerecycled.
Wanneer je haast hebt dus niet checkt of er wc papier hangt in het toilet en je komt erachter dat het er niet is 
Goed bezig Bente! Lekker de aandacht afleiden van jouw corrupte ambtsgenoten! Volgens mij denk je ook nog dat niemand dat doorziet! Graag de vragen printen op rollen 

## Store as dataframe and drop duplicates

In [7]:
import re
def delete_hashtags(tweet):
    return re.sub('#', '', tweet)

tweets_df = DataFrame([[tweet.user_id, delete_hashtags(tweet.text), tweet.retweets, tweet.timestamp] for tweet in tweets], columns=['user_id', 'text', 'retweets', 'timestamp'])

In [8]:
tweets_df = tweets_df.drop_duplicates()

## Translate to english for generela sentiment analysis

In [9]:
from googletrans import Translator

In [10]:
translator = Translator()
tweets_text = tweets_df['text'].tolist()
translations = translator.translate(tweets_text)
translations = [x.text for x in translations]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
tweets_df['translation'] = translations

## Sentiment analysis

In [ ]:
# https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
def add_translation(row):
    text = row['text']
    translation = translator.translate(text).text

    row['translation'] = translation

    return row

def sentiment_analysis(row):
    trans = row['translation']
    sentiment = sid.polarity_scores(trans)

    row['negative'] = sentiment['neg']
    row['neutral'] = sentiment['neu']
    row['positive'] = sentiment['pos']

    return row




In [ ]:
tweets_df_with_sentiment = tweets_df.apply(sentiment_analysis, axis=1)

In [ ]:
tweets_df_with_sentiment

## Weekly analysis

In [ ]:
tweets_df_with_sentiment.set_index(tweets_df_with_sentiment['timestamp'])[['negative', 'neutral', 'positive']].resample('W').sum()

## MOnthly analysis

In [ ]:
tweets_df_with_sentiment.set_index(tweets_df_with_sentiment['timestamp'])[['negative', 'neutral', 'positive']].resample('M').sum()

In [ ]:
tweets_df_per_period = tweets_df_with_sentiment.set_index(tweets_df_with_sentiment['timestamp'])[['negative', 'neutral', 'positive']].resample('M').sum()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
neg = np.array(tweets_df_per_period['negative'].to_list())
neutr = np.array(tweets_df_per_period['neutral'].to_list())
pos = np.array(tweets_df_per_period['positive'].to_list())

neg_neutr = neg + neutr
neg_neutr_pos = neg_neutr + pos


In [ ]:
# %b for month, %d for day, %y for year 
dates = [x.strftime("%b") for x in tweets_df_per_period.index.tolist()]

fig = plt.figure()
plt.plot(dates, neg, 'r')
plt.plot(dates, neg_neutr, 'orange')
plt.plot(dates, neg_neutr_pos, 'g')
plt.fill_between(dates, [0 for x in neg], neg, facecolor='red', alpha=0.5)
plt.fill_between(dates, neg, neg_neutr, facecolor='orange', alpha=0.5)
plt.fill_between(dates, neg_neutr, neg_neutr_pos, facecolor='green', alpha=0.5)